# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Stored Procedures](#stored-procedures)**
3. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/common/schemas.ipynb",
        "src/static/cbs_inhabitants_per_age_group.ipynb"
    ]
}
```

# **TABLES**

---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE TABLE(S).....
DROP TABLE IF EXISTS [VWSSTAGE].[CBS_AGE_GROUP_MAPPING]

CREATE TABLE [VWSSTAGE].[CBS_AGE_GROUP_MAPPING] (
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE(),
	[KEY] [VARCHAR](100) NULL,
	[TITLE] [VARCHAR](100) NULL,
	[DESCRIPTION] [VARCHAR](100) NULL,
	[CATEGORY_GROUP_ID] [VARCHAR](100) NULL
);

-- 2) CREATE STAGE INDEX(ES).....
CREATE NONCLUSTERED INDEX [NCIX_DLI_CBS_AGE_GROUP_MAPPING] ON [VWSSTAGE].[CBS_AGE_GROUP_MAPPING](
    [DATE_LAST_INSERTED] ASC
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STATIC TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_AGE_GROUP_MAPPING]'))
CREATE TABLE [VWSSTATIC].[CBS_AGE_GROUP_MAPPING] (
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[KEY] [INT] NULL,
	[TITLE] [VARCHAR](100) NULL,
	[DESCRIPTION] [VARCHAR](100) NULL,
	[CATEGORY_GROUP_ID] [INT] NULL
);
GO

-- 2) CREATE STATIC INDEX(ES).....
IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_CBS_AGE_GROUP_MAPPING')
CREATE NONCLUSTERED INDEX [NCIX_DLI_CBS_AGE_GROUP_MAPPING] ON [VWSSTATIC].[CBS_AGE_GROUP_MAPPING]
(
    [DATE_LAST_INSERTED] ASC
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STATIC TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_AGE_GROUP_MAPPING_DISTRIBUTION]'))
CREATE TABLE [VWSSTATIC].[CBS_AGE_GROUP_MAPPING_DISTRIBUTION] (
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE(),
	[AGE_GROUP] [VARCHAR](10) NULL,
	[INHABITANTS] [BIGINT] NULL,
	[INHABITANTS_PERCENTAGE] [DECIMAL](10,3) NULL
);
GO

-- 2) CREATE STATIC INDEX(ES).....
IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_CBS_AGE_GROUP_MAPPING_DISTRIBUTION')
CREATE NONCLUSTERED INDEX [NCIX_DLI_CBS_AGE_GROUP_MAPPING_DISTRIBUTION] ON [VWSSTATIC].[CBS_AGE_GROUP_MAPPING_DISTRIBUTION] (
    [DATE_LAST_INSERTED] ASC
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

# **STORED PROCEDURES**

---

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_CBS_AGE_GROUP_MAPPING_STAGE_TO_STATIC]
AS
BEGIN

    SELECT
        CAST([KEY] AS INT) AS [KEY],
        [TITLE],
        CASE
            WHEN [CATEGORY_GROUP_ID] = 2 THEN '0-9'
            WHEN [CATEGORY_GROUP_ID] = 3 THEN '10-19'
            WHEN [CATEGORY_GROUP_ID] = 4 THEN '20-29'
            WHEN [CATEGORY_GROUP_ID] = 5 THEN '30-39'
            WHEN [CATEGORY_GROUP_ID] = 6 THEN '40-49'
            WHEN [CATEGORY_GROUP_ID] = 7 THEN '50-59'
            WHEN [CATEGORY_GROUP_ID] = 8 THEN '60-69'
            WHEN [CATEGORY_GROUP_ID] = 9 THEN '70-79'
            WHEN [CATEGORY_GROUP_ID] = 10 THEN '80-89'
            WHEN [CATEGORY_GROUP_ID] > 10 AND [TITLE] <> '95 JAAR OF OUDER' THEN '90+'
        END AS [DESCRIPTION],
        CAST([CATEGORY_GROUP_ID] AS INT) AS [CATEGORY_GROUP_ID]
    INTO #CBS_AGE_GROUP_MAPPING_BASE
    FROM [VWSSTAGE].[CBS_AGE_GROUP_MAPPING]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_AGE_GROUP_MAPPING]);

    -- 1) INSERT INTO AGE GROUP MAPPING
    INSERT INTO [VWSSTATIC].[CBS_AGE_GROUP_MAPPING] ([KEY], [TITLE], [DESCRIPTION], [CATEGORY_GROUP_ID])
    SELECT 
        [KEY], 
        [TITLE], 
        [DESCRIPTION], 
        [CATEGORY_GROUP_ID]
    FROM #CBS_AGE_GROUP_MAPPING_BASE;

    -- 2) INSERT INTO AGE GROUP DISTRIBUTION
    WITH SELECTED_RECORDS AS (
        SELECT 
            cagm.[DESCRIPTION],
            cipag.[POPULATION_ON_FIRST_OF_JANUARY_1],
            cipag.[DATE_LAST_INSERTED]
        FROM [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP] cipag
        LEFT JOIN #CBS_AGE_GROUP_MAPPING_BASE cagm ON cipag.[AGE] = cagm.[KEY]
        WHERE cipag.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP])
    ),
    AGGREGATED_RECORDS AS (
        SELECT 
            [DESCRIPTION] AS [AGE_GROUP],
            SUM([POPULATION_ON_FIRST_JANUARY_1]) AS [INHABITANTS]
        FROM SELECTED_RECORDS
        GROUP BY [DESCRIPTION]
    )
    INSERT INTO [VWSSTATIC].[CBS_AGE_GROUP_MAPPING_DISTRIBUTION] ([AGE_GROUP], [INHABITANTS], [INHABITANTS_PERCENTAGE])
    SELECT 
        [AGE_GROUP],
        [INHABITANTS],
        -- DIVIDING THE POPULATION NUMBERS PER AGE GROUP BY THE TOTAL AMOUNT OF THE POPULATION ACROSS ALL AGE GROUPS TO RECEIVE THE RESPECTIVE PERCENTAGE
        CAST((CAST([INHABITANTS] AS DECIMAL(15,3))/(SELECT SUM([INHABITANTS]) FROM AGGREGATED_RECORDS WHERE [AGE_GROUP] IS NOT NULL)) AS DECIMAL(10,3))
    FROM AGGREGATED_RECORDS;

    -- 3) DROP TABLES OR RECORDS
    DROP TABLE IF EXISTS #CBS_AGE_GROUP_MAPPING_BASE;

    TRUNCATE TABLE [VWSSTAGE].[CBS_AGE_GROUP_MAPPING];
    
END;
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'CBS_AGE_GROUP_MAPPING',
        @workflow_static_name NVARCHAR(50),
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @workflow_static_name = CONCAT(N'STATIC_', @workflow_name);
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_static_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_static_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_static_name,
    @description = @workflow_description,
    @schedule = '0 12 * * *', -- AT 01:30 PM EVERYDAY
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'https://opendata.cbs.nl/ODataApi/odata/03759ned/Leeftijd'
    WHEN 'acceptance' THEN 'https://opendata.cbs.nl/ODataApi/odata/03759ned/Leeftijd'
    ELSE 'https://opendata.cbs.nl/ODataApi/odata/03759ned/Leeftijd'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'Web'
    WHEN 'acceptance' THEN 'Web'
    ELSE 'Web'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Key|Title|Description|CategoryGroupID',
    @target_columns = 'KEY|TITLE|DESCRIPTION|CATEGORY_GROUP_ID',
    @target_name = @target_name,
    @source_type = 'JsonFile',
    @location_type = @location_type,
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source = CONCAT('dbo.SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO STATIC');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = CONCAT('PROCESS_', @workflow_name);
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_static_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_static_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO STATIC......
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO STATIC');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_STATIC');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_static_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_static_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_STATIC');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_static_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_static_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO

-- 4.2) UPSERT EXTERNAL STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES).....
DECLARE @external_dependency VARCHAR(50) = 'CBS_INHABITANTS_PER_AGE_GROUP';
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_WITH_', @external_dependency);
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_STATIC');
SET @dependency_process_name = CONCAT('PROCESS_SP_', @external_dependency, '_STAGE_TO_STATIC');
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_static_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_static_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO